# Micro_organisms_classification

In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import splitfolders

## Data Preparation

In [122]:
input_dir='E:\\DATA Science\\DEEP LEARNING\\CNN\\Micro_organisms_classification\\Dataset\\Micro_Organism'
output_dir='E:\\DATA Science\\DEEP LEARNING\\CNN\\Micro_organisms_classification\\Dataset_split'

In [123]:
#splitfolders.ratio(input=input_dir,output=output_dir,ratio=(0.7,0.2,0.1))

In [124]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [125]:
train_gen=ImageDataGenerator(rescale=1/255,rotation_range=20,horizontal_flip=True,
                             width_shift_range=0.2,height_shift_range=0.2)
test_gen=ImageDataGenerator(rescale=1/255)

In [126]:
train_dir='E:\\DATA Science\\DEEP LEARNING\\CNN\\Micro_organisms_classification\\Dataset_split\\train'
test_dir='E:\\DATA Science\\DEEP LEARNING\\CNN\\Micro_organisms_classification\\Dataset_split\\test'
val_dir='E:\\DATA Science\\DEEP LEARNING\\CNN\\Micro_organisms_classification\\Dataset_split\\val'

In [127]:
train_itt=train_gen.flow_from_directory(train_dir,target_size=(256, 256))
val_itt=test_gen.flow_from_directory(val_dir,target_size=(256, 256))
test_itt=test_gen.flow_from_directory(test_dir,target_size=(256, 256))

Found 549 images belonging to 8 classes.
Found 156 images belonging to 8 classes.
Found 84 images belonging to 8 classes.


## CNN Model

In [128]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten

In [129]:
model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())

model.add(Dense(512,activation='relu'))

model.add(Dense(8,activation='softmax'))

In [130]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(train_itt,validation_data=val_itt,epochs=8)

Epoch 1/8
 1/18 [>.............................] - ETA: 2:10 - loss: 2.0684 - accuracy: 0.2500

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score,accuracy_score

In [ ]:
predictions = model.evaluate(test_itt)

In [ ]:
losses = pd.DataFrame(model.history.history)

## Dropouts

In [ ]:
model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(512,activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(8,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(train_itt,validation_data=val_itt,epochs=8)

In [ ]:
predictions = model.evaluate(test_itt)

## Padding

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(256,256,3),padding='same'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())

model.add(Dense(512,activation='relu'))

model.add(Dense(8,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(train_itt,validation_data=val_itt,epochs=8)

In [ ]:
predictions = model.evaluate(test_itt)

## Tunning

In [ ]:
import keras_tuner

In [ ]:
def build_model(hp):
    model =Sequential()
    model.add(Conv2D(32,(3,3),activation='relu',input_shape=(256,256,3)))
    model.add(MaxPooling2D((2,2)))

    model.add(Conv2D(64,(3,3),activation='relu'))
    model.add(MaxPooling2D((2,2)))

    model.add(Conv2D(128,(3,3),activation='relu'))
    model.add(MaxPooling2D((2,2)))

    model.add(Flatten())

    model.add(Dense(units=hp.Choice('layes size',[64,128,256,512]),activation='relu'))

    model.add(Dense(8,activation='softmax')) 
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return model

In [ ]:
tuner = keras_tuner.RandomSearch(hypermodel=build_model,
    objective="val_accuracy",
    max_trials=4,
    overwrite=True)

In [ ]:
tuner.search_space_summary()
tuner.search(train_itt,validation_data=test_itt,epochs=2)

In [ ]:
best_model=tuner.get_best_models()[0]

In [ ]:
best_model.evaluate(test_itt)

In [ ]:
class_label= test_itt.class_indices